In [2]:
%pip install scikit-learn numpy matplotlib networkx pandas scipy torch statsmodels


Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-2.0.0-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
print(torch.version.cuda)

11.7


In [3]:
%pip install pyg_lib torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-1.13.0+cu117.html
# tu trzeba wpisać odpowiednią wersję CUDA, patrz: https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html

Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu117.html

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install torch-geometric

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 KB 5.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached psutil-5.9.5-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (282 kB)
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=895986 sha256=374f18a950d6f45b04b7572519b3133d73cccfe7f83f1ea5b589159bb1023344
  Stored in directory: /home/szymonr/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric
Note: you may need to restart the kernel to use updated packages.


In [5]:
from torch_geometric.datasets import TUDataset

ROOT_DATA_PATH = "../data"

imdb_binary_dataset = TUDataset(root=ROOT_DATA_PATH, name="IMDB-BINARY")
imdb_multi_dataset = TUDataset(root=ROOT_DATA_PATH, name="IMDB-MULTI")
reddit_binary_dataset = TUDataset(root=ROOT_DATA_PATH, name="REDDIT-BINARY")
reddit_multi_5k_dataset = TUDataset(root=ROOT_DATA_PATH, name="REDDIT-MULTI-5K")
reddit_multi_12k_dataset = TUDataset(root=ROOT_DATA_PATH, name="REDDIT-MULTI-12K")

# do pobrania jeszcze dataset z cząsteczkami stąd: https://ogb.stanford.edu/docs/graphprop/#pyg

Extracting ../data/IMDB-BINARY/IMDB-BINARY.zip
Processing...
Done!
Extracting ../data/IMDB-MULTI/IMDB-MULTI.zip
Processing...
Done!
Extracting ../data/REDDIT-BINARY/REDDIT-BINARY.zip
Processing...
Done!
Extracting ../data/REDDIT-MULTI-5K/REDDIT-MULTI-5K.zip
Processing...
Done!
Extracting ../data/REDDIT-MULTI-12K/REDDIT-MULTI-12K.zip
Processing...
Done!


In [12]:
HYPERPARAMS = {
    'n_bin': 30,  # number of bins for historgram
    'norm_flag': 'no',  # normalize before calling function_basis versus normalize after
    'nonlinear_flag': 'True',  # SVM linear kernel versus nonlinear kernel
    'uniform_flag': True,  # unform versus log scale. True for imdb, False for reddit.
    'cdf_flag': True,  # cdf versus pdf. True for most dataset.
    'his_norm_flag': 'yes'
}

In [13]:
from main import evaluate

evaluate(dataset="imdb_binary", classifier='svm', baseline='ldp', hyperparams=HYPERPARAMS)

Loading graph from dataset imdb_binary
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
Number of bins are 30
the shape after removing zero columns is  (1000, 145)
imdb_binary
Evaluating with SVM
[0.77 0.73 0.75 0.78 0.69 0.69 0.67 0.76 0.74 0.69]
[0.72 0.77 0.69 0.73 0.74 0.64 0.79 0.69 0.76 0.78]
[0.71 0.73 0.73 0.69 0.7  0.73 0.75 0.74 0.76 0.77]
[0.71 0.72 0.74 0.71 0.8  0.71 0.67 0.69 0.76 0.71]
[0.71 0.75 0.78 0.73 0.77 0.71 0.7  0.7  0.69 0.73]
[0.77 0.74 0.68 0.77 0.73 0.72 0.81 0.72 0.65 0.64]
[0.7  0.68 0.75 0.68 0.79 0.71 0.73 0.67 0.78 0.66]
[0.73 0.71 0.73 0.81 0.68 0.76 0.7  0.73 0.64 0.72]
[0.74 0.79 0.66 0.76 0.69 0.74 0.77 0.76 0.7  0.67]
[0.68 0.66 0.8  0.7  0.78 0.75 0.75 0.7  0.72 0.76]
mean is 0.7254999999999999, std is 0.00490407993409567 
'evaluate_svm'  7.68 s


In [15]:
from main import evaluate

evaluate(dataset="imdb_binary", classifier='svm', baseline='ldp_extended', hyperparams=HYPERPARAMS)

Loading graph from dataset imdb_binary
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
Number of bins are 30
The max of list is 2.614135177033117


AssertionError: 

In [14]:
from graph import load_graph
from main import convert_graphs_to_vectors

dataset = 'imdb_binary'

graphs, labels = load_graph(dataset)

x_ldp, y_ldp = convert_graphs_to_vectors(dataset, graphs, labels, baseline='ldp', hyperparams=HYPERPARAMS)

print(f"LDP x: {x_ldp.shape}, y: {y_ldp.shape}")

x_invars, y_invars = convert_graphs_to_vectors(dataset, graphs, labels, baseline='graph_invariants', hyperparams=HYPERPARAMS)

print(f"Graph invariants x: {x_invars.shape}, y: {y_invars.shape}")

Loading graph from dataset imdb_binary
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
Number of bins are 30
the shape after removing zero columns is  (1000, 145)
LDP x: (1000, 145), y: (1000,)
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
Graph invariants x: (1000, 4), y: (1000,)


In [8]:
evaluate(dataset="imdb_binary", classifier='random_forest', baseline='ldp', hyperparams=HYPERPARAMS)

Loading graph from dataset imdb_binary
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
Number of bins are 30
the shape after removing zero columns is  (1000, 145)
Evaluating with Random Forest
Accuracy: 0.715


In [15]:
evaluate(dataset="imdb_binary", classifier='svm', baseline='graph_invariants', hyperparams=HYPERPARAMS)

Loading graph from dataset imdb_binary
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
imdb_binary
Evaluating with SVM
[0.64 0.61 0.61 0.61 0.69 0.64 0.67 0.64 0.65 0.64]
[0.65 0.66 0.63 0.57 0.71 0.59 0.63 0.65 0.64 0.66]
[0.7  0.53 0.68 0.69 0.64 0.6  0.68 0.66 0.62 0.65]
[0.64 0.71 0.65 0.59 0.68 0.62 0.65 0.68 0.62 0.58]
[0.69 0.66 0.65 0.66 0.7  0.66 0.59 0.61 0.6  0.63]
[0.71 0.66 0.65 0.6  0.62 0.55 0.65 0.72 0.59 0.63]
[0.54 0.59 0.65 0.61 0.64 0.67 0.72 0.73 0.66 0.62]
[0.71 0.58 0.7  0.63 0.58 0.63 0.66 0.59 0.7  0.61]
[0.66 0.67 0.55 0.67 0.63 0.6  0.73 0.57 0.66 0.69]
[0.61 0.53 0.62 0.67 0.75 0.71 0.63 0.65 0.63 0.6 ]
mean is 0.6414, std is 0.002416609194718913 
'evaluate_svm'  118.57 s


In [10]:
evaluate(dataset="IMDB-BINARY", classifier='random_forest', baseline='ldp', hyperparams=HYPERPARAMS)

READ GRAPH
GRAPH READED
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
Number of bins are 30
the shape after removing zero columns is  (1000, 145)
Evaluating with Random Forest
Accuracy: 0.725
